# 1. class KNN

## 1-1 class KNN 만들기

충전소 설립 후보 지역의 위경도를 기준으로 KNN을 찾은 뒤 거리에 따라 가중치를 부여하여 특성변수를 만들어주는 class 만들기

In [4]:
class KNN:
    import pandas as pd
    import numpy as np
    # station_xy : 충전소를 세울 예정이 있는 곳의 위경도 데이터(위도,경도순으로 input받아야함)
    # neighbor_xy : KNN으로 찾아지는 neighbor들의 위경도 데이터
    # K : KNN에서의 K 값
    # df : KNN으로 찾아지는 neighbor들이 가지고 있는 특성변수에 대한 정보값
    def __init__(self, station_xy, K, neighbor_xy):
        self.station_xy = station_xy
        self.K = K
        self.neighbor_xy = neighbor_xy
    
    # KNN을 찾아주는 함수
    def KNN_search(self):
        
        # self 활용
        station_xy = self.station_xy.copy()
        neighbor_xy = self.neighbor_xy.copy()
        K = self.K
        
        # 함수에서 사용을 위해 neighbor_xy의 첫번째 열을 gid로 바꿔주기
        # neighbor_xy = neighbor_xy.rename(columns={0:'gid'})
    
        KNN_result = station_xy.reset_index(drop=True)
        for i in range(K) :
            KNN_result['neighbor'+str(i)]=np.nan
            KNN_result['weight'+str(i)]=np.nan
        
        for i in range(len(station_xy)):

            # 각 격자와 충전소의 거리정보를 저장할 임시 데이터 프레임 dists를 만듬
            dists = neighbor_xy.copy() #명칭, 위도, 경도 순으로 받아야함
            
            stationY=station_xy.iloc[i,0]    # y:위도
            stationX=station_xy.iloc[i,1]    # x:경도
      
            #새로운 데이터의 좌표와 알고있는 모든 데이터 집합의 좌표 사이 거리 계산
            dists['위도차'] = dists.iloc[:,1] - stationY
            dists['경도차'] = dists.iloc[:,2] - stationX
    
            dists['거리'] = dists['경도차']**2 + dists['위도차']**2
    
            dists.sort_values(by='거리', axis=0, ascending=True, inplace=True)
    
            select_neighbor = dists.iloc[0:K,:].copy()
            
            select_neighbor['거리역수'] = 1/select_neighbor['거리']
            select_neighbor['가중치'] = select_neighbor['거리역수']/sum(select_neighbor['거리역수'])
            
            select_neighbor = select_neighbor[['gid', '가중치']]
    
            for j in range(K):
                KNN_result.iloc[i,2*j+2]=select_neighbor.iloc[j,0]
                KNN_result.iloc[i,2*j+3]=select_neighbor.iloc[j,1]
    
        return KNN_result


    
    # KNN 결과의 가중치를 활용하여 특성변수를 가중평균하여 충전소별로 배치하는 함수
    def KNN_sum(self, df): 
        
        KNN_result = self.KNN_search()
        
        # 함수에서 사용을 위해 df의 첫번째 열을 gid로 바꿔주기
        # df = df.rename(columns={0:'gid'})

        # 결과값을 저장할 빈 데이터 프레임 만들기
        final_result = pd.DataFrame()
    
        for i in range(len(KNN_result)):
            # KNN 값을 합칠 빈 데이터 프레임 만들기
            merge_KNN = pd.DataFrame()
        
            # 상위 K개 포인트에 대한 특성변수 정보 불러오기
            for j in range(int((KNN_result.shape[1]-2)/2)):
                temp = df[df['gid']==KNN_result.iloc[i,2+2*j]]
                merge_KNN = merge_KNN.append(temp, ignore_index = True)
        
            # 'gid' 행 삭제
            merge_KNN = merge_KNN.drop(['gid'], axis = 1)
        
            # 행마다 가중치 부여
            for j in range(int((KNN_result.shape[1]-2)/2)):
                merge_KNN.iloc[j,:]=merge_KNN.iloc[j,:]*KNN_result.iloc[i,3+2*j]
    
            # 가중치 부여한 행들 합산
            merge_KNN=merge_KNN.sum(axis=0)
    
            # merge_KNN을 final_result에 한 행씩 저장
            final_result = final_result.append(merge_KNN,ignore_index=True)
    
        final_result.insert(0, '경도', KNN_result.iloc[:,1])
        final_result.insert(0, '위도', KNN_result.iloc[:,0])

        return final_result

## 1-2 class KNN 설명

총 3개의 데이터프레임과 1개의 정수를 입력값으로 받는다. class안의 함수 두개를 통해 최종적으로 final_result를 출력하는 것을 목표로 한다.

### 1-2-1 입력값(input)

1) station_xy

충전소를 세울 예정이 있는 곳의 위경도 데이터(데이터프레임)
주의!! 위도 경도 순으로 input 되어야 함

2) neighbor_xy
KNN으로 찾아지는 neighbor들의 위경도 데이터(데이터프레임)
주의!! 첫행의 name을 'gid'로 바꾸어 주어야 하고 첫 행에 들어있는 정보가 중복되지 않고 각 지역을 데표 할 수 있어야 함

3) K
KNN에서의 K 값(정수)

4) df : KNN으로 찾아지는 neighbor들이 가지고 있는 특성변수에 대한 정보 데이터(데이터프레임)
주의) 첫행의 name이 'gid'이어야 함

### 1-2-2 KNN_search() 함수

station_xy, neighbor_xy, K를 활용

station_xy에 있는 위경도 쌍 각각에 대해서 아래의 작업을 수행

1) station_xy에 있는 위경도 쌍과 neighbor_xy에 있는 위경도쌍을 활용하여 거리를
   구하고 가장 가까운 K개의 지역을 neighbor_xy에서 찾는다.

2) 찾은 K개의 지역에 대하여 거리 역수로 가중치를 준다.

3) 찾은 K개의 지역과 가중치를 KNN_result 데이터프레임에 저장한다.

station_xy에 있는 모든 위경도쌍에 대하여 위의 작업을 수행한 후 KNN_result를
출력한다.

### 1-2-3 KNN_sum() 함수

KNN_result와 df를 활용

KNN_result의 지역과 가중치를 활용하여 df에 있는 특성변수에 대한 가중합을 구한다.

가중합 된 특성변수를 final_result로 출력하고 각 충전소 설립 후보지역의 특성변수로 활용한다.

# 2. 함수 활용에 필요한 데이터 받기

In [24]:
import pandas as pd
import numpy as np

## 2-1 격자데이터 : seoul_data

In [25]:
seoul_data=pd.read_csv('dat2.csv',encoding='cp949')
seoul_data = seoul_data.drop(columns='Unnamed: 0')
seoul_data

,gid,y,x,pop,wpop,apop,b_ratio,f_ratio,num_b,r_area,land_price
0,다사3550,37.551586,126.769783,0,0,0,13.90,18.41,10,0.0,185658.59
1,다사3551,37.560599,126.769695,0,0,0,39.95,46.51,2,116.4,185686.53
2,다사3552,37.569612,126.769607,0,0,0,0.00,0.00,0,0.0,55693.74
3,다사3649,37.542643,126.781189,0,0,0,0.00,0.00,0,0.0,147119.68
4,다사3650,37.551656,126.781103,0,0,0,9.22,30.62,4,0.0,202491.93
...,...,...,...,...,...,...,...,...,...,...,...
705,다사7151,37.562417,127.177297,15346,10696,3102,48.34,317.74,198,0.0,3505548.10
706,다사7152,37.571430,127.177258,13070,8651,2342,54.24,213.48,146,0.0,5539614.55
707,다사7153,37.580444,127.177220,0,0,0,0.00,0.00,0,0.0,216147.16
708,다사7249,37.544421,127.188695,0,0,0,0.00,0.00,0,0.0,0.00


In [26]:
# 격자 중심점 위경도 데이터
seoul_data_xy = seoul_data[['gid','y','x']]
seoul_data_xy

,gid,y,x
0,다사3550,37.551586,126.769783
1,다사3551,37.560599,126.769695
2,다사3552,37.569612,126.769607
3,다사3649,37.542643,126.781189
4,다사3650,37.551656,126.781103
...,...,...,...
705,다사7151,37.562417,127.177297
706,다사7152,37.571430,127.177258
707,다사7153,37.580444,127.177220
708,다사7249,37.544421,127.188695


In [27]:
# 위경도 데이터 없애기
seoul_data = seoul_data.drop(columns=['y', 'x'])
seoul_data

,gid,pop,wpop,apop,b_ratio,f_ratio,num_b,r_area,land_price
0,다사3550,0,0,0,13.90,18.41,10,0.0,185658.59
1,다사3551,0,0,0,39.95,46.51,2,116.4,185686.53
2,다사3552,0,0,0,0.00,0.00,0,0.0,55693.74
3,다사3649,0,0,0,0.00,0.00,0,0.0,147119.68
4,다사3650,0,0,0,9.22,30.62,4,0.0,202491.93
...,...,...,...,...,...,...,...,...,...
705,다사7151,15346,10696,3102,48.34,317.74,198,0.0,3505548.10
706,다사7152,13070,8651,2342,54.24,213.48,146,0.0,5539614.55
707,다사7153,0,0,0,0.00,0.00,0,0.0,216147.16
708,다사7249,0,0,0,0.00,0.00,0,0.0,0.00


## 2-2 교통량데이터 : traffic_data

In [30]:
traffic_data = pd.read_csv('traffic.csv',encoding='cp949')
traffic_data = traffic_data.drop(columns='Unnamed: 0')
traffic_data

,지점번호,일통행량,지점명칭,위도,경도,주소
0,A-01,27314.696429,성산로(금화터널),37.568588,126.948436,서울시 서대문구 신촌동 1-142
1,A-02,30035.216667,사직로(사직터널),37.572298,126.962853,서울시 종로구 행촌동 1-186
2,A-03,11734.633333,자하문로(자하문터널),37.588831,126.968548,서울시 종로구 청운동 24-6
3,A-04,4861.338889,대사관로(삼청터널),37.596359,126.984209,서울시 성북구 성북동 산25-148
4,A-05,23828.178571,율곡로(안국역),37.576000,126.984342,서울시 종로구 경운동 90-4
...,...,...,...,...,...,...
133,F-06,90510.017045,경부고속도로,37.493135,127.022533,서울시 서초구 서초동 1748-26
134,F-07,62665.976608,분당수서로,37.497648,127.087195,서울시 강남구 일원동 467-1
135,F-08,37882.420455,강남순환로(관악터널),37.449096,126.926169,서울시 금천구 시흥동 산 93-1
136,F-09,20506.755556,서부간선지하도로,37.468942,126.883673,서울시 금천구 가산동 338-10


In [31]:
# 교통량 포인트 위경도 데이터
traffic_data_xy = traffic_data[['지점명칭','위도','경도']]
traffic_data_xy = traffic_data_xy.rename(columns={'지점명칭':'gid'})
traffic_data_xy

,gid,위도,경도
0,성산로(금화터널),37.568588,126.948436
1,사직로(사직터널),37.572298,126.962853
2,자하문로(자하문터널),37.588831,126.968548
3,대사관로(삼청터널),37.596359,126.984209
4,율곡로(안국역),37.576000,126.984342
...,...,...,...
133,경부고속도로,37.493135,127.022533
134,분당수서로,37.497648,127.087195
135,강남순환로(관악터널),37.449096,126.926169
136,서부간선지하도로,37.468942,126.883673


In [32]:
# 필요한 데이터만 뽑아내고 행순서 바꾸기
traffic_data = traffic_data[['지점명칭', '일통행량']]
traffic_data = traffic_data.rename(columns={'지점명칭':'gid'})
traffic_data

,gid,일통행량
0,성산로(금화터널),27314.696429
1,사직로(사직터널),30035.216667
2,자하문로(자하문터널),11734.633333
3,대사관로(삼청터널),4861.338889
4,율곡로(안국역),23828.178571
...,...,...
133,경부고속도로,90510.017045
134,분당수서로,62665.976608
135,강남순환로(관악터널),37882.420455
136,서부간선지하도로,20506.755556


## 2-3 동별 자동차 등록대수 데이터 : elect_car_data

In [34]:
elect_car_data=pd.read_excel('동별 자동차 등록대수.xlsx')
elect_car_data

,addr,num,X,Y,행정동명,법정동명
0,서울특별시 강남구 개포1동,24,127.057498,37.481978,개포1동,NaN
1,서울특별시 강남구 개포2동,121,127.069137,37.489819,개포2동,NaN
2,서울특별시 강남구 개포4동,71,127.051624,37.478850,개포4동,NaN
3,서울특별시 강남구 논현1동,75,127.028555,37.511469,논현1동,NaN
4,서울특별시 강남구 논현2동,102,127.037359,37.517389,논현2동,NaN
...,...,...,...,...,...,...
422,서울특별시 중랑구 상봉2동,36,127.080876,37.592982,상봉2동,NaN
423,서울특별시 중랑구 신내1동,100,127.099495,37.605866,신내1동,NaN
424,서울특별시 중랑구 신내2동,76,127.094179,37.606240,신내2동,NaN
425,서울특별시 중랑구 중화1동,30,127.080674,37.601171,중화1동,NaN


In [35]:
# 행정동 데이터에 결측 확인
elect_car_data[elect_car_data['행정동명'].isnull()==True]

,addr,num,X,Y,행정동명,법정동명
31,서울특별시 강동구 상일제1동,69,127.123872,37.530119,NaN,NaN
32,서울특별시 강동구 상일제2동,11,127.123872,37.530119,NaN,NaN
220,서울특별시 서대문구 신촌동,30,126.939285,37.565160,NaN,신촌동


In [36]:
# 결측치 채워주기
elect_car_data.iloc[31,4] = '상일제1동'
elect_car_data.iloc[32,4] = '상일제2동'
elect_car_data.iloc[220,4] ='신촌동'

elect_car_data[elect_car_data['행정동명'].isnull()==True]

,addr,num,X,Y,행정동명,법정동명


In [37]:
elect_car_data

,addr,num,X,Y,행정동명,법정동명
0,서울특별시 강남구 개포1동,24,127.057498,37.481978,개포1동,NaN
1,서울특별시 강남구 개포2동,121,127.069137,37.489819,개포2동,NaN
2,서울특별시 강남구 개포4동,71,127.051624,37.478850,개포4동,NaN
3,서울특별시 강남구 논현1동,75,127.028555,37.511469,논현1동,NaN
4,서울특별시 강남구 논현2동,102,127.037359,37.517389,논현2동,NaN
...,...,...,...,...,...,...
422,서울특별시 중랑구 상봉2동,36,127.080876,37.592982,상봉2동,NaN
423,서울특별시 중랑구 신내1동,100,127.099495,37.605866,신내1동,NaN
424,서울특별시 중랑구 신내2동,76,127.094179,37.606240,신내2동,NaN
425,서울특별시 중랑구 중화1동,30,127.080674,37.601171,중화1동,NaN


In [38]:
# 동별 위경도 데이터
elect_car_data_xy = elect_car_data[['행정동명','Y','X']]
elect_car_data_xy = elect_car_data_xy.rename(columns={'행정동명':'gid'})
elect_car_data_xy

,gid,Y,X
0,개포1동,37.481978,127.057498
1,개포2동,37.489819,127.069137
2,개포4동,37.478850,127.051624
3,논현1동,37.511469,127.028555
4,논현2동,37.517389,127.037359
...,...,...,...
422,상봉2동,37.592982,127.080876
423,신내1동,37.605866,127.099495
424,신내2동,37.606240,127.094179
425,중화1동,37.601171,127.080674


In [39]:
# 필요한 데이터만 뽑아내고 행순서 바꾸기
elect_car_data = elect_car_data[['행정동명',' num ']] #num 양옆에 한칸 띄우기
elect_car_data = elect_car_data.rename(columns={'행정동명':'gid'})
elect_car_data

,gid,num
0,개포1동,24
1,개포2동,121
2,개포4동,71
3,논현1동,75
4,논현2동,102
...,...,...
422,상봉2동,36
423,신내1동,100
424,신내2동,76
425,중화1동,30


# 3. 충전소 설립 후보지역 위경도 데이터에 KNN class 실제 적용

## 3-1. 충전소 설립 후보지역 데이터 받기

In [42]:
sample_data = pd.read_csv('sample.csv',encoding='cp949')
sample_data = sample_data[['y', 'x']]
sample_data

,y,x
0,37.479132,127.097849
1,37.646592,127.046026
2,37.521695,127.067788
3,37.545310,127.186173
4,37.637137,127.068117
5,37.584924,126.967558
6,37.615603,126.997116
7,37.628316,127.081170
8,37.584854,127.000622
9,37.602030,126.970806


## 3-2. KNN class 적용

### 3-1. 격자 데이터에 적용

In [43]:
KNN_fn_seoul_data = KNN(station_xy=sample_data, neighbor_xy=seoul_data_xy, K=4)
result = KNN_fn_seoul_data.KNN_search()
result

,y,x,neighbor0,weight0,neighbor1,weight1,neighbor2,weight2,neighbor3,weight3
0,37.479132,127.097849,다사6442,0.870206,다사6441,0.069998,다사6342,0.030171,다사6443,0.029625
1,37.646592,127.046026,다사5960,0.335573,다사6060,0.252497,다사5961,0.225824,다사6061,0.186107
2,37.521695,127.067788,다사6147,0.376024,다사6146,0.340034,다사6247,0.145172,다사6246,0.138770
3,37.545310,127.186173,다사7249,0.806221,다사7250,0.079910,다사7149,0.073705,다사7150,0.040164
4,37.637137,127.068117,다사6159,0.422036,다사6259,0.218116,다사6160,0.213948,다사6260,0.145901
5,37.584924,126.967558,다사5254,0.287867,다사5354,0.274563,다사5253,0.223791,다사5353,0.213779
6,37.615603,126.997116,다사5557,0.939840,다사5556,0.022494,다사5558,0.020025,다사5657,0.017641
7,37.628316,127.081170,다사6358,0.349061,다사6258,0.268923,다사6359,0.207072,다사6259,0.174944
8,37.584854,127.000622,다사5554,0.337786,다사5553,0.271639,다사5654,0.209702,다사5653,0.180873
9,37.602030,126.970806,다사5355,0.423740,다사5356,0.369953,다사5255,0.105258,다사5256,0.101049


In [45]:
grid_feature=KNN_fn_seoul_data.KNN_sum(df=seoul_data)

### 3-2. 교통량 데이터에 적용

In [46]:
KNN_fn_traffic_data = KNN(station_xy=sample_data, neighbor_xy=traffic_data_xy, K=2)
result = KNN_fn_traffic_data.KNN_search()
result

,y,x,neighbor0,weight0,neighbor1,weight1
0,37.479132,127.097849,헌릉로(세곡동사거리),0.561359,남부순환로(수서IC),0.438641
1,37.646592,127.046026,도봉로(쌍문역),0.609377,동일로(노원역),0.390623
2,37.521695,127.067788,청담대교,0.865638,영동대교,0.134362
3,37.545310,127.186173,천호대로(상일IC),0.920315,올림픽대로(강일IC),0.079685
4,37.637137,127.068117,동부간선도로(월계1교),0.842861,동일로(노원역),0.157139
5,37.584924,126.967558,자하문로(자하문터널),0.917880,사직로(사직터널),0.082120
6,37.615603,126.997116,내부순환로,0.867266,평창문화로(북악터널),0.132734
7,37.628316,127.081170,화랑로(화랑대역),0.814925,화랑로(조선왕릉),0.185075
8,37.584854,127.000622,창경궁로(서울여자대학교),0.791620,대학로(한국방송통신대학교),0.208380
9,37.602030,126.970806,평창문화로(북악터널),0.524027,자하문로(자하문터널),0.475973


In [47]:
traffic_feature=KNN_fn_traffic_data.KNN_sum(df=traffic_data)

### 3-3. 전기차 대수 데이터에 적용

In [48]:
KNN_fn_elect_car_data = KNN(station_xy=sample_data, neighbor_xy=elect_car_data_xy, K=3)
result = KNN_fn_elect_car_data.KNN_search()
result

,y,x,neighbor0,weight0,neighbor1,weight1,neighbor2,weight2
0,37.479132,127.097849,일원본동,0.389881,수서동,0.389362,일원1동,0.220758
1,37.646592,127.046026,창3동,0.398507,창5동,0.322331,창1동,0.279161
2,37.521695,127.067788,삼성1동,0.502542,자양4동,0.262485,자양3동,0.234973
3,37.545310,127.186173,상일1동,0.467471,강일동,0.303029,고덕2동,0.229500
4,37.637137,127.068117,중계2.3동,0.357243,하계2동,0.352570,하계1동,0.290187
5,37.584924,126.967558,청운효자동,0.780960,부암동,0.116925,사직동,0.102115
6,37.615603,126.997116,정릉3동,0.443492,정릉4동,0.409147,수유1동,0.147361
7,37.628316,127.081170,공릉2동,0.475432,공릉1동,0.388617,하계2동,0.135951
8,37.584854,127.000622,혜화동,0.875751,성북동,0.077657,이화동,0.046592
9,37.602030,126.970806,평창동,0.794381,부암동,0.143825,청운효자동,0.061794


In [49]:
elect_car_feature=KNN_fn_elect_car_data.KNN_sum(df=elect_car_data)

### 3-4. 만들어진 3종류 feature 데이터 병합하여 최종 feature data 산출

In [50]:
merge_1 = pd.merge(grid_feature, traffic_feature, on=['위도', '경도'], how='inner')
merge_1

,위도,경도,pop,wpop,apop,b_ratio,f_ratio,num_b,r_area,land_price,일통행량
0,37.479132,127.097849,3827.020187,2553.526693,902.323075,39.182602,74.349468,201.143414,242.889409,1.297801e+06,34201.288933
1,37.646592,127.046026,22238.353103,15768.294532,4727.825445,46.325230,179.649482,858.964844,120.452779,2.318498e+06,18292.392851
2,37.521695,127.067788,4949.715990,3532.441308,854.651590,32.342597,153.473467,201.871178,149.461725,5.039565e+06,56815.334307
3,37.545310,127.186173,485.711019,344.300227,86.143243,5.726151,19.044918,16.561466,27.328600,2.664864e+05,49030.292609
4,37.637137,127.068117,28813.838691,20398.723464,6000.902260,46.057104,149.616114,300.106125,133.760610,2.721819e+06,54123.720703
5,37.584924,126.967558,4858.731907,3404.328353,1017.612827,48.277993,123.616187,822.665787,147.615023,2.031905e+06,13237.472693
6,37.615603,126.997116,4149.508096,2712.855298,1207.990717,43.073741,80.158936,355.364025,84.396440,1.342113e+06,60007.518203
7,37.628316,127.081170,19106.228638,14035.718865,3441.689748,43.626945,130.837018,712.249710,188.135294,2.419967e+06,29844.195620
8,37.584854,127.000622,15977.441900,11798.273931,3177.894468,56.092353,168.696914,1972.044687,136.093661,3.856633e+06,19486.764813
9,37.602030,126.970806,3788.623294,2705.360111,775.178178,36.918601,84.320613,383.984761,148.178991,1.437731e+06,15388.621005


In [51]:
merge_2 = pd.merge(merge_1, elect_car_feature, on=['위도', '경도'], how='inner')
merge_2

,위도,경도,pop,wpop,apop,b_ratio,f_ratio,num_b,r_area,land_price,일통행량,num
0,37.479132,127.097849,3827.020187,2553.526693,902.323075,39.182602,74.349468,201.143414,242.889409,1.297801e+06,34201.288933,52.975765
1,37.646592,127.046026,22238.353103,15768.294532,4727.825445,46.325230,179.649482,858.964844,120.452779,2.318498e+06,18292.392851,37.369624
2,37.521695,127.067788,4949.715990,3532.441308,854.651590,32.342597,153.473467,201.871178,149.461725,5.039565e+06,56815.334307,88.778021
3,37.545310,127.186173,485.711019,344.300227,86.143243,5.726151,19.044918,16.561466,27.328600,2.664864e+05,49030.292609,111.341743
4,37.637137,127.068117,28813.838691,20398.723464,6000.902260,46.057104,149.616114,300.106125,133.760610,2.721819e+06,54123.720703,36.516347
5,37.584924,126.967558,4858.731907,3404.328353,1017.612827,48.277993,123.616187,822.665787,147.615023,2.031905e+06,13237.472693,59.063032
6,37.615603,126.997116,4149.508096,2712.855298,1207.990717,43.073741,80.158936,355.364025,84.396440,1.342113e+06,60007.518203,36.626255
7,37.628316,127.081170,19106.228638,14035.718865,3441.689748,43.626945,130.837018,712.249710,188.135294,2.419967e+06,29844.195620,61.048045
8,37.584854,127.000622,15977.441900,11798.273931,3177.894468,56.092353,168.696914,1972.044687,136.093661,3.856633e+06,19486.764813,24.786106
9,37.602030,126.970806,3788.623294,2705.360111,775.178178,36.918601,84.320613,383.984761,148.178991,1.437731e+06,15388.621005,74.030204


In [52]:
merge_2.to_csv('충전소 설립 후보 지역에 대한 KNN 결과.csv')